In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="uda-mlws")
exp = Experiment(workspace=ws, name="azuremlexp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: uda-mlws
Azure region: westus2
Subscription id: 5a1f52fe-4556-47f9-a53e-ede8e50a6980
Resource group: uda-mlrg


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "azure-ml-cluster"

compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True,min_node_count = 1, timeout_in_minutes = 5)
print(aml_compute.get_status().serialize())

Succeeded.........................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-10-11T09:26:33.062000+00:00', 'errors': None, 'creationTime': '2020-10-11T09:26:27.380777+00:00', 'modifiedTime': '2020-10-11T09:26:43.081356+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
params = {
    'C': uniform(0.1,0.3),
    'max_iter':choice(25,50,75)
}
ps = RandomParameterSampling(params)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training", compute_target=aml_compute, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy',
                 estimator=est, policy=policy, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                  max_total_runs=4, max_concurrent_runs=4 )

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdriverun = exp.submit(hyperdrive_config, show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [11]:
hyperdriverun

Experiment,Id,Type,Status,Details Page,Docs Page
azuremlexp,HD_313e5278-e13b-4b3f-8eb1-5e846a606682,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
RunDetails(hyperdriverun).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [25]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdriverun.get_best_run_by_primary_metric()
joblib.dump(best_run, 'best_hyperdrive_model.sav')

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['accuracy'])

Best Run Id: HD_313e5278-e13b-4b3f-8eb1-5e846a606682

 Accuracy: 0.915254640256903


In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dataset_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path=dataset_link)

In [17]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [18]:
# Submit your automl run

automl_run = exp.submit(automl_config,show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on local machine
Parent Run ID: AutoML_86bfbe9b-b924-4f1a-b92c-4c71123b6639

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |yes                              |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_86bfbe9b-b924-4f1a-b92c-4c71123b6639',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-10-11T18:42:57.403895Z',
 'endTimeUtc': '2020-10-11T19:19:26.654715Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"azuremlexp","subscription_id":"5a1f52fe-4556-47f9-a53e-ede8e50a6980","resource_group":"uda-mlrg","workspace_name":"uda-mlws","region":"westus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":2,"featurization":"auto","_ignore_package_version

In [21]:
# Retrieve and save your best automl model.

best_automl_run, fitted_model = automl_run.get_output()
best_run_metrics = best_automl_run.get_metrics()

print('\n Accuracy:', best_run_metrics['accuracy'])


 Accuracy: 0.9179362670713201


In [22]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.01,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=10,
                     

In [23]:
joblib.dump(fitted_model, 'best_automl_model.joblib')

['best_automl_model.joblib']

In [26]:
AmlCompute.delete(aml_compute)

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

